In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import math
import copy
os.chdir('C:/Users/user/Desktop/프로젝트')
warnings.filterwarnings('ignore')

In [2]:
from haversine import haversine

## 격자 별 공공체육시설 최소 거리

In [3]:
data_1=pd.read_csv('거주인구_위도경도.csv',encoding='cp949')
data_reset_popindex = copy.deepcopy(data_1)
data_reset_popindex=data_reset_popindex[['CTY_NM','X_AXIS','Y_AXIS','경도','위도','POP','POP_10','POP_20','POP_30','POP_40','POP_50','POP_60_O']]
data_reset_popindex['min_dis']=0
gong=pd.read_csv('서울시 공공체육시설 현황(2019).csv',encoding='cp949')
def method_1(data_reset_popindex,gong):
    l1=len(gong)
    l2=len(data_reset_popindex)
    gong_x=list(gong['위도'].astype('float'))
    gong_y=list(gong['경도'].astype('float'))
    data_x=list(data_reset_popindex['위도'].astype('float'))
    data_y=list(data_reset_popindex['경도'].astype('float'))
    for i in range(l2):
        a=(data_x[i],data_y[i])
        min_dis=9999999999999999
        for j in range(l1):
            b=(gong_x[j],gong_y[j])
            dis=haversine(a,b, unit = 'm')
            if min_dis>dis:
                min_dis=dis
        data_reset_popindex.iloc[i,12]=min_dis

    return data_reset_popindex

In [4]:
data_reset_pop = method_1(data_reset_popindex,gong)

* 거리 칼럼 추가

In [5]:
data_reset_pop

,CTY_NM,X_AXIS,Y_AXIS,경도,위도,POP,POP_10,POP_20,POP_30,POP_40,POP_50,POP_60_O,min_dis
0,동작구,307050,544550,126.945794,37.495686,446.33,109.34,50.29,95.34,65.09,62.81,63.46,590.842822
1,동작구,309850,543750,126.977553,37.488756,446.21,75.99,85.91,78.27,70.74,72.35,62.95,247.142918
2,동작구,308350,545350,126.960394,37.503023,128.32,17.52,29.28,16.37,21.75,19.60,23.80,578.184570
3,동작구,305550,544150,126.928884,37.491930,374.03,69.40,53.73,78.98,53.09,60.78,58.05,366.870891
4,동작구,306450,544850,126.938971,37.498328,452.06,75.69,70.77,92.50,68.39,72.52,72.19,767.001922
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29653,은평구,304150,557750,126.911278,37.614311,478.90,104.71,70.62,80.12,81.94,74.76,66.75,535.597496
29654,은평구,303750,556550,126.906906,37.603459,402.31,113.48,65.10,74.72,72.35,49.57,27.09,1101.272455
29655,관악구,305450,543050,126.927895,37.482009,60.58,3.58,15.87,13.20,8.36,8.08,11.49,557.694127
29656,서초구,313850,544450,127.022698,37.495444,269.18,41.36,42.13,21.89,45.68,77.85,40.27,355.719463


* 연령별 참여도 가중치(설문조사 데이터)

In [6]:
pop_sum=data_reset_pop['POP_10']*0.073+data_reset_pop['POP_20']*0.1219+data_reset_pop['POP_30']*0.214+data_reset_pop['POP_40']*0.247+data_reset_pop['POP_50']*0.293+data_reset_pop['POP_60_O']*0.287
data_reset_pop['NEW_POP'] = pop_sum
data_reset_pop['dis_pop']=data_reset_pop['min_dis']*data_reset_pop['NEW_POP']

* 구별 공공체육시설 평균 거리(가중평균)
* 가장 높은 강서구에 대한 공공체육시설 입지 분석

In [7]:
data_group=data_reset_pop.groupby('CTY_NM').sum()
group_pop=data_group['POP_10']*0.073+data_group['POP_20']*0.1219+data_group['POP_30']*0.214+data_group['POP_40']*0.247+data_group['POP_50']*0.293+data_group['POP_60_O']*0.287
per_dis_list=data_group['dis_pop']/group_pop
per_dis_list=per_dis_list.sort_values(ascending=False)
per_dis_list

CTY_NM
강서구     796.380817
송파구     689.050129
종로구     687.114190
은평구     660.708061
강북구     617.066274
금천구     611.980317
구로구     608.799838
중랑구     602.668244
관악구     571.007878
동대문구    569.623019
강남구     567.829230
서초구     566.213991
성북구     545.718993
서대문구    535.439437
강동구     535.279568
동작구     533.010087
용산구     526.942141
광진구     522.537712
영등포구    514.223261
양천구     511.854449
마포구     473.013522
노원구     464.601064
성동구     441.560252
도봉구     430.066510
중구      327.341232
dtype: float64

In [8]:
school=pd.read_csv('서울시 초, 중, 고 학교 위치.csv',encoding='cp949')  ##강서구 학교 
school=school[school['시도교육청명']=='서울특별시교육청']
gangsu=school[school['소재지지번주소'].apply(lambda x: '강서구' in x)].reset_index(drop=True)

data_1=pd.read_csv('거주인구_위도경도.csv',encoding='cp949')
data_pop = copy.deepcopy(data_1)
data_pop=data_pop[data_pop['CTY_NM']=='강서구']
data_pop=data_pop[['CTY_NM','X_AXIS','Y_AXIS','경도','위도','POP','POP_10','POP_20','POP_30','POP_40','POP_50','POP_60_O']]
data_pop['min_dis']=0

* 순차적으로 평균거리가 가장 짧아지도록 하는 초, 중, 고 추천

In [9]:
def phase(gu,data_pop,gong):
    new_dis=[]
    for i in range(len(gu)):
        gu_1=gu[['학교명','위도','경도']].loc[i]
        gu_1=pd.DataFrame(gu_1).T
        gu_1.columns=['시설명','위도','경도']
        new_data=pd.concat([gong,gu_1])
        new_data=new_data.reset_index(drop=True)
        data_new=method_1(data_pop,new_data)
        pop_new=data_new['POP_10']*0.073+data_new['POP_20']*0.1219+data_new['POP_30']*0.214+data_new['POP_40']*0.247+data_new['POP_50']*0.293+data_new['POP_60_O']*0.287
        data_new['NEW_POP'] = pop_new
        data_new['dis_pop']=data_new['min_dis']*data_new['NEW_POP']

        dis_sum=data_new.groupby('CTY_NM').sum()['dis_pop']
        pop_df=data_new.groupby('CTY_NM').sum()
        pop_sum=pop_df['POP_10']*0.073+pop_df['POP_20']*0.1219+pop_df['POP_30']*0.214+pop_df['POP_40']*0.247+pop_df['POP_50']*0.293+pop_df['POP_60_O']*0.287
        per_dis_1=dis_sum/pop_sum
        new_dis.append(per_dis_1.values[0])
    new_phase=pd.concat([gu[['학교명','위도','경도']],pd.DataFrame(new_dis,columns=['new_dis'])],axis=1)
    new_phase=new_phase.sort_values(by='new_dis').reset_index(drop=True)
    return new_phase

In [11]:
school=pd.read_csv('서울시 초, 중, 고 학교 위치.csv',encoding='cp949')  ##강서구 학교 
school=school[school['시도교육청명']=='서울특별시교육청']
gangsu=school[school['소재지지번주소'].apply(lambda x: '강서구' in x)].reset_index(drop=True)

data_1=pd.read_csv('거주인구_위도경도.csv',encoding='cp949')
data_pop = copy.deepcopy(data_1)
data_pop=data_pop[data_pop['CTY_NM']=='강서구']
data_pop=data_pop[['CTY_NM','X_AXIS','Y_AXIS','경도','위도','POP','POP_10','POP_20','POP_30','POP_40','POP_50','POP_60_O']]
data_pop['min_dis']=0

phase1=phase(gangsu,data_pop,gong)

In [12]:
plus=pd.DataFrame(phase1[['학교명','위도','경도']].loc[0]).T
plus.columns=['시설명','위도','경도']
new_gong=pd.concat([gong,plus])
new_gong=new_gong.reset_index(drop=True)

school_name=phase1[['학교명']].loc[0][0]
gangsu=gangsu[gangsu['학교명']!=school_name].reset_index(drop=True)
                   
data_pop = copy.deepcopy(data_1)
data_pop=data_pop[data_pop['CTY_NM']=='강서구']
data_pop=data_pop[['CTY_NM','X_AXIS','Y_AXIS','경도','위도','POP','POP_10','POP_20','POP_30','POP_40','POP_50','POP_60_O']]
data_pop['min_dis']=0

gangsu[['학교명','위도','경도']]

phase2=phase(gangsu,data_pop,new_gong)

In [13]:
plus=pd.DataFrame(phase2[['학교명','위도','경도']].loc[0]).T
plus.columns=['시설명','위도','경도']
new_gong=pd.concat([new_gong,plus])
new_gong=new_gong.reset_index(drop=True)

school_name=phase2[['학교명']].loc[0][0]
gangsu=gangsu[gangsu['학교명']!=school_name].reset_index(drop=True)
                   
data_pop = copy.deepcopy(data_1)
data_pop=data_pop[data_pop['CTY_NM']=='강서구']
data_pop=data_pop[['CTY_NM','X_AXIS','Y_AXIS','경도','위도','POP','POP_10','POP_20','POP_30','POP_40','POP_50','POP_60_O']]
data_pop['min_dis']=0
gangsu[['학교명','위도','경도']]

phase3=phase(gangsu,data_pop,new_gong)

In [14]:
plus=pd.DataFrame(phase3[['학교명','위도','경도']].loc[0]).T
plus.columns=['시설명','위도','경도']
new_gong=pd.concat([new_gong,plus])
new_gong=new_gong.reset_index(drop=True)

school_name=phase3[['학교명']].loc[0][0]
gangsu=gangsu[gangsu['학교명']!=school_name].reset_index(drop=True)       

data_pop = copy.deepcopy(data_1)
data_pop=data_pop[data_pop['CTY_NM']=='강서구']
data_pop=data_pop[['CTY_NM','X_AXIS','Y_AXIS','경도','위도','POP','POP_10','POP_20','POP_30','POP_40','POP_50','POP_60_O']]
data_pop['min_dis']=0
gangsu[['학교명','위도','경도']]

phase4=phase(gangsu,data_pop,new_gong)

In [15]:
plus=pd.DataFrame(phase4[['학교명','위도','경도']].loc[0]).T
plus.columns=['시설명','위도','경도']
new_gong=pd.concat([new_gong,plus])
new_gong=new_gong.reset_index(drop=True)

school_name=phase4[['학교명']].loc[0][0]
gangsu=gangsu[gangsu['학교명']!=school_name].reset_index(drop=True)
                   
data_pop = copy.deepcopy(data_1)
data_pop=data_pop[data_pop['CTY_NM']=='강서구']
data_pop=data_pop[['CTY_NM','X_AXIS','Y_AXIS','경도','위도','POP','POP_10','POP_20','POP_30','POP_40','POP_50','POP_60_O']]
data_pop['min_dis']=0
gangsu[['학교명','위도','경도']]

phase5=phase(gangsu,data_pop,new_gong)

In [16]:
plus=pd.DataFrame(phase5[['학교명','위도','경도']].loc[0]).T
plus.columns=['시설명','위도','경도']
new_gong=pd.concat([new_gong,plus])
new_gong=new_gong.reset_index(drop=True)

school_name=phase5[['학교명']].loc[0][0]
gangsu=gangsu[gangsu['학교명']!=school_name].reset_index(drop=True)
                   
data_pop = copy.deepcopy(data_1)
data_pop=data_pop[data_pop['CTY_NM']=='강서구']
data_pop=data_pop[['CTY_NM','X_AXIS','Y_AXIS','경도','위도','POP','POP_10','POP_20','POP_30','POP_40','POP_50','POP_60_O']]
data_pop['min_dis']=0
gangsu[['학교명','위도','경도']]

phase6=phase(gangsu,data_pop,new_gong)

In [17]:
plus=pd.DataFrame(phase6[['학교명','위도','경도']].loc[0]).T
plus.columns=['시설명','위도','경도']
new_gong=pd.concat([new_gong,plus])
new_gong=new_gong.reset_index(drop=True)

school_name=phase6[['학교명']].loc[0][0]
gangsu=gangsu[gangsu['학교명']!=school_name].reset_index(drop=True)
                   
data_pop = copy.deepcopy(data_1)
data_pop=data_pop[data_pop['CTY_NM']=='강서구']
data_pop=data_pop[['CTY_NM','X_AXIS','Y_AXIS','경도','위도','POP','POP_10','POP_20','POP_30','POP_40','POP_50','POP_60_O']]
data_pop['min_dis']=0
gangsu[['학교명','위도','경도']]

phase7=phase(gangsu,data_pop,new_gong)

In [18]:
plus=pd.DataFrame(phase7[['학교명','위도','경도']].loc[0]).T
plus.columns=['시설명','위도','경도']
new_gong=pd.concat([new_gong,plus])
new_gong=new_gong.reset_index(drop=True)

school_name=phase7[['학교명']].loc[0][0]
gangsu=gangsu[gangsu['학교명']!=school_name].reset_index(drop=True)
                   
data_pop = copy.deepcopy(data_1)
data_pop=data_pop[data_pop['CTY_NM']=='강서구']
data_pop=data_pop[['CTY_NM','X_AXIS','Y_AXIS','경도','위도','POP','POP_10','POP_20','POP_30','POP_40','POP_50','POP_60_O']]
data_pop['min_dis']=0
gangsu[['학교명','위도','경도']]

phase8=phase(gangsu,data_pop,new_gong)


In [19]:
plus=pd.DataFrame(phase8[['학교명','위도','경도']].loc[0]).T
plus.columns=['시설명','위도','경도']
new_gong=pd.concat([new_gong,plus])
new_gong=new_gong.reset_index(drop=True)

school_name=phase8[['학교명']].loc[0][0]
gangsu=gangsu[gangsu['학교명']!=school_name].reset_index(drop=True)
                   
data_pop = copy.deepcopy(data_1)
data_pop=data_pop[data_pop['CTY_NM']=='강서구']
data_pop=data_pop[['CTY_NM','X_AXIS','Y_AXIS','경도','위도','POP','POP_10','POP_20','POP_30','POP_40','POP_50','POP_60_O']]
data_pop['min_dis']=0
gangsu[['학교명','위도','경도']]

phase9=phase(gangsu,data_pop,new_gong)


In [20]:
plus=pd.DataFrame(phase9[['학교명','위도','경도']].loc[0]).T
plus.columns=['시설명','위도','경도']
new_gong=pd.concat([new_gong,plus])
new_gong=new_gong.reset_index(drop=True)

school_name=phase9[['학교명']].loc[0][0]
gangsu=gangsu[gangsu['학교명']!=school_name].reset_index(drop=True)
                   
data_pop = copy.deepcopy(data_1)
data_pop=data_pop[data_pop['CTY_NM']=='강서구']
data_pop=data_pop[['CTY_NM','X_AXIS','Y_AXIS','경도','위도','POP','POP_10','POP_20','POP_30','POP_40','POP_50','POP_60_O']]
data_pop['min_dis']=0
gangsu[['학교명','위도','경도']]

phase10=phase(gangsu,data_pop,new_gong)


In [21]:
plus=pd.DataFrame(phase10[['학교명','위도','경도']].loc[0]).T
plus.columns=['시설명','위도','경도']
new_gong=pd.concat([new_gong,plus])
new_gong=new_gong.reset_index(drop=True)

school_name=phase10[['학교명']].loc[0][0]
gangsu=gangsu[gangsu['학교명']!=school_name].reset_index(drop=True)
                   
data_pop = copy.deepcopy(data_1)
data_pop=data_pop[data_pop['CTY_NM']=='강서구']
data_pop=data_pop[['CTY_NM','X_AXIS','Y_AXIS','경도','위도','POP','POP_10','POP_20','POP_30','POP_40','POP_50','POP_60_O']]
data_pop['min_dis']=0
gangsu[['학교명','위도','경도']]

phase11=phase(gangsu,data_pop,new_gong)


In [25]:
gangsu_total=pd.concat([pd.DataFrame(phase1.loc[0]).T,pd.DataFrame(phase2.loc[0]).T,pd.DataFrame(phase3.loc[0]).T,
                       pd.DataFrame(phase4.loc[0]).T,pd.DataFrame(phase5.loc[0]).T,pd.DataFrame(phase6.loc[0]).T,
                       pd.DataFrame(phase7.loc[0]).T,pd.DataFrame(phase8.loc[0]).T,pd.DataFrame(phase9.loc[0]).T,
                       pd.DataFrame(phase10.loc[0]).T,pd.DataFrame(phase11.loc[0]).T])

In [26]:
gangsu_total

,학교명,위도,경도,new_dis
0,서울신정초등학교,37.5321,126.852,650.4
0,서울신월초등학교,37.5404,126.839,604.528
0,덕원여자고등학교,37.5486,126.829,572.053
0,서울방화초등학교,37.5687,126.807,540.702
0,서울염창초등학교,37.5532,126.868,514.95
0,서울등명초등학교,37.5609,126.842,490.384
0,화원중학교,37.5362,126.842,467.56
0,서울등촌초등학교,37.5476,126.858,448.324
0,성재중학교,37.5695,126.845,432.816
0,서울화일초등학교,37.5407,126.847,418.087


In [27]:
gangsu_total['ph']=['p1','p2','p3','p4','p5','p6','p7','p8','p9','p10','p11']

In [28]:
gangsu_total.to_csv('강서구 11개의 학교 추천.csv',encoding='cp949')

* 설문조사 전처리 데이터 불러오기


## 독립변수 설명
- f0 성별
    * 1:남자
    * 2:여자
  
  
* f1 연령
    * 1 : 0 ~ 18
    * 2 : 19 ~ 29
    * 3 : 30 ~ 49
    * 4 : 50 ~ 64
    * 5 : 64 ~ 90
    
    
* f2 본인의 건강상태에 대한 인식
    * 0 : 부정
    * 1 : 긍정
    
    
* f3 최근 1년간 규칙적 체육활동 참여 빈도
    * 0 : 일주일에 1 ~ 3번 & 한달에 3번(부정)
    * 1 : 일주일에 4 ~ 7번(긍정)
    
    
* f4 최근 1개월간 신체적·정신적 문제로 인한 지장 여부
    * 0 : 없음
    * 1 : 있음
    
    
* f5 학력
    * 0 : 대졸미만
    * 1: 대졸이상
    
    
* f6 가구원수
    * 1 : 1명
    * 2 : 2명
    * 3 : 3명이상

In [36]:
people=pd.read_csv('인원result.csv',encoding='cp949')
tool=pd.read_csv('도구result.csv',encoding='cp949')
sisul=pd.read_csv('시설result.csv',encoding='cp949')
sisul['class2']=people['class']
sisul['class3']=tool['class']
data=sisul

data=data.astype('category')
data.head()

,f0,f1,f2,f3,f4,f5,f6,class,class2,class3
0,1,1,0,1,1,0,3,106,120,116
1,1,1,1,0,1,0,3,120,120,116
2,1,1,1,0,0,0,3,120,120,409
3,1,1,1,0,1,0,3,106,103,106
4,1,1,1,0,1,0,3,120,120,409


### class 설명
* class(시설)
    * 120 : 실내
    * 106 : 실외
    
    
* class2(인원)
    * 116 : 1인
    * 120 : 2인
    * 103 : 3인 이상
    
    
* class3(도구 사용)
    * 116 : 도구 사용
    * 103 : 손
    * 106 : 발
    * 409 : 전신

### Y 설명
* 종목
    * 103 : 농구
    * 106 : 축구,풋살
    * 107 : 족구
    * 116 : 테니스,정구,스퀴시,라켓볼
    * 202 : 검도
    * 405 : 수영
    * 407 : 에어로빅
    * 120 : 탁구
    * 117 : 배드민턴
    * 409 : 댄스 스포츠
    * 408 : 요가, 필라테스, 태보

In [37]:
data_y=pd.read_csv('원본.csv',encoding='cp949')

data_y

data['y']=data_y['class']
data['y']=data['y'].astype('category')

data.head()

,f0,f1,f2,f3,f4,f5,f6,class,class2,class3,y
0,1,1,0,1,1,0,3,106,120,116,116
1,1,1,1,0,1,0,3,120,120,116,117
2,1,1,1,0,0,0,3,120,120,409,405
3,1,1,1,0,1,0,3,106,103,106,106
4,1,1,1,0,1,0,3,120,120,409,408


### MLP class1

In [116]:
#MLP 정규화
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler(feature_range=(0,1))

features=['f{}'.format(x) for x in range(0,7)]

train_d,test_d=train_test_split(data,test_size=0.3)

train_y=train_d['class']
train_x=train_d[features]
train_x=scaler.fit_transform(train_x)

test_y=test_d['class']
test_x=test_d[features]
test_x=scaler.fit_transform(test_x)

model_MLP1=MLPClassifier(max_iter=500)
model_MLP1.fit(train_x,train_y)

model_MLP1.score(test_x,test_y)

0.817283950617284

### MLP class2

In [126]:
scaler=MinMaxScaler(feature_range=(0,1))

features=['f{}'.format(x) for x in range(0,7)]

train_d,test_d=train_test_split(data,test_size=0.3)

train_y=train_d['class2']
train_x=train_d[features]
train_x=scaler.fit_transform(train_x)

test_y=test_d['class2']
test_x=test_d[features]
test_x=scaler.fit_transform(test_x)

model_MLP2=MLPClassifier(max_iter=500)
model_MLP2.fit(train_x,train_y)

model_MLP2.score(test_x,test_y)

0.8682098765432099

### MLP class3

In [127]:
scaler=MinMaxScaler(feature_range=(0,1))

features=['f{}'.format(x) for x in range(0,7)]

train_d,test_d=train_test_split(data,test_size=0.3)

train_y=train_d['class3']
train_x=train_d[features]
train_x=scaler.fit_transform(train_x)

test_y=test_d['class3']
test_x=test_d[features]
test_x=scaler.fit_transform(test_x)

model_MLP3=MLPClassifier(max_iter=500)
model_MLP3.fit(train_x,train_y)

model_MLP3.score(test_x,test_y)

0.7225308641975309

### Gradient Boosting class1

In [119]:
from sklearn.ensemble import GradientBoostingClassifier

train_d,test_d=train_test_split(data,test_size=0.3)

train_y=train_d['class']
train_x=train_d[features]

test_y=test_d['class']
test_x=test_d[features]

model_GB1=GradientBoostingClassifier()
model_GB1.fit(train_x,train_y)

model_GB1.score(test_x,test_y)

0.817283950617284

### Gradient Boosting class2

In [120]:
train_d,test_d=train_test_split(data,test_size=0.3)

train_y=train_d['class2']
train_x=train_d[features]

test_y=test_d['class2']
test_x=test_d[features]

model_GB2=GradientBoostingClassifier()
model_GB2.fit(train_x,train_y)

model_GB2.score(test_x,test_y)

0.8765432098765432

### Gradient Boosting class3

In [121]:
train_d,test_d=train_test_split(data,test_size=0.3)

train_y=train_d['class3']
train_x=train_d[features]

test_y=test_d['class3']
test_x=test_d[features]

model_GB3=GradientBoostingClassifier()
model_GB3.fit(train_x,train_y)

model_GB3.score(test_x,test_y)

0.7108024691358025

### RandomForest class1

In [133]:
from sklearn.ensemble import RandomForestClassifier


train_d,test_d=train_test_split(data,test_size=0.3)

train_y=train_d['class']
train_x=train_d[features]

test_y=test_d['class']
test_x=test_d[features]

model_RF1=RandomForestClassifier(n_estimators=100, oob_score=True, random_state=456,min_samples_leaf=10)
model_RF1.fit(train_x,train_y)

model_RF1.score(test_x,test_y)

0.8265432098765432

### RandomForest class2

In [123]:
train_d,test_d=train_test_split(data,test_size=0.3)

train_y=train_d['class2']
train_x=train_d[features]

test_y=test_d['class2']
test_x=test_d[features]

model_RF2=RandomForestClassifier(n_estimators=100, oob_score=True, random_state=456,min_samples_leaf=10)
model_RF2.fit(train_x,train_y)

model_RF2.score(test_x,test_y)

0.8870370370370371

### RandomForest class3

In [125]:
train_d,test_d=train_test_split(data,test_size=0.3)

train_y=train_d['class3']
train_x=train_d[features]

test_y=test_d['class3']
test_x=test_d[features]

model_RF3=RandomForestClassifier(n_estimators=100, oob_score=True, random_state=456,min_samples_leaf=10)
model_RF3.fit(train_x,train_y)

model_RF3.score(test_x,test_y)

0.7283950617283951

* MLP Accuracy
    * class1 : 0.817283950617284
    * class2 : 0.8682098765432099
    * class3 : 0.7225308641975309


* Gradient Boosting Accuracy
    * class1 : 0.817283950617284
    * class2 : 0.8765432098765432
    * class3 : 0.7108024691358025
    
    
* RandomForest Accuracy
    * class1 : 0.8265432098765432
    * class2 : 0.8870370370370371
    * class3 : 0.7283950617283951
    

* 속도가 빠르고, 정확도 높은 랜덤포레스트 모델 선택

## 종목 추천 Decision Tree Model

In [135]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz

train_d,test_d=train_test_split(data,test_size=0.3)

train_y=train_d['y']
train_x=train_d[['class','class2','class3']]

test_y=test_d['y']
test_x=test_d[['class','class2','class3']]

model_DT=DecisionTreeClassifier(max_depth=5,random_state=100)

model_DT=model_DT.fit(train_x,train_y)

model_DT.score(test_x,test_y)

#export_graphviz.fit(model,out_file)

#export_graphviz(model,out_file='시각화.html',impurity=False,filled=True)

#!pip insat

#(graph,)=pydot.graph_from_dot_file('시각화.',encoding='utf8')
#graph.write_png('시각화.jpg')

#model.score(train_x,train_y)

0.5743827160493827

* 서울 신정초등학교 화곡 2동 인구 샘플링 데이터

In [97]:
sample_data=pd.read_csv('화곡2동.csv',encoding='cp949')
sample_data=sample_data.astype('category')
sample_data['ex']=1

In [99]:
group=sample_data.groupby(['성별', '연령대', '건강인식여부', '규칙적운동여부', '정신건강여부', '학력', '가구원수']).sum()

In [102]:
group.sort_values(by='ex',ascending=False)

ex
성별 연령대 건강인식여부 규칙적운동여부 정신건강여부 학력 가구원수       
2  3   0      0       1      0  3     207.0
1  3   0      0       1      0  3     180.0
2  3   0      0       1      1  3     167.0
1  3   0      0       1      1  3     155.0
2  4   0      0       1      0  3     150.0
...                                     ...
   5   0      1       0      1  1       NaN
       1      1       0      0  1       NaN
                                2       NaN
                                3       NaN
                             1  1       NaN

[480 rows x 1 columns]

* 화곡 2동에 가장 많은 인구 유형(1순위)

In [155]:
sample1=list(group.sort_values(by='ex',ascending=False).iloc[0,:-1].name)
sample1=pd.DataFrame(sample1).astype('category').T

In [167]:
sample1_class1=model_RF1.predict(sample1)
sample1_class2=model_RF2.predict(sample1)
sample1_class3=model_RF3.predict(sample1)
sample1_df=pd.DataFrame([sample1_class1,sample1_class2,sample1_class3]).astype('category').T

* 서울신정초등학교(화곡 2동) 수영 종목 추천

In [168]:
model_DT.predict(sample1_df)

array([405], dtype=int64)

* 화곡 2동에 두 번째로 많은 인구 유형(2순위)

In [169]:
sample2=list(group.sort_values(by='ex',ascending=False).iloc[1,:-1].name)
sample2=pd.DataFrame(sample2).astype('category').T
sample2_class1=model_RF1.predict(sample2)
sample2_class2=model_RF2.predict(sample2)
sample2_class3=model_RF3.predict(sample2)
sample2_df=pd.DataFrame([sample2_class1,sample2_class2,sample2_class3]).astype('category').T

* 서울신정초등학교(화곡 2동) 수영 종목 추천

In [170]:
model_DT.predict(sample2_df)

array([405], dtype=int64)

* 화곡 2동에 세 번째로 많은 인구 유형(3순위)

In [171]:
sample3=list(group.sort_values(by='ex',ascending=False).iloc[2,:-1].name)
sample3=pd.DataFrame(sample3).astype('category').T
sample3_class1=model_RF1.predict(sample3)
sample3_class2=model_RF2.predict(sample3)
sample3_class3=model_RF3.predict(sample3)
sample3_df=pd.DataFrame([sample3_class1,sample3_class2,sample3_class3]).astype('category').T

* 서울신정초등학교(화곡 2동) 수영 종목 추천

In [172]:
model_DT.predict(sample3_df)

array([405], dtype=int64)

* 1순위, 2순위, 3순위 인구 유형 모두 수영 종목 추천

## 결론
* 1순위 구 : 강서구
* 최적 입지 선정 : 서울신정초등학교(화곡2동)
* 종목 추천 : 수영